In [4]:
# Install anomalib and other dependencies
!pip install anomalib
!pip install pytorch-lightning==2.3.0

import torch
import numpy as np
import matplotlib.pyplot as plt
from anomalib.data import MVTecAD
from anomalib.models import Patchcore
from anomalib.engine import Engine
# from anomalib.utils.visualization import get_visualization_image # Commented out due to ImportError

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.6/831.6 kB 69.3 MB/s eta 0:00:00
  Created wheel for freia: filename=FrEIA-0.2-py3-none-any.whl size=42763 sha256=a41f637bc4ec8f09af8860cfd5638d368a59d36e3ac67c3d93668929241b5972
  Stored in 

In [ ]:
from anomalib.data import MVTecAD
from torchvision import transforms
import anomalib
print(anomalib.__version__)
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.listdir("/content/drive/MyDrive/anomalib_data/MVTec"))

datamodule = MVTecAD(
    root="/content/drive/MyDrive/anomalib_data/MVTec",
    category="carpet"
)
datamodule.setup()


# Initialize the PatchCore model with a pre-trained WideResNet50 backbone.
model = Patchcore(
    backbone="wide_resnet50_2",
    layers=["layer2", "layer3"],
    coreset_sampling_ratio=0.1,
)
#
engine = Engine(
   # pixel_metrics=["pro", "aupro"],
   # image_metrics=["auroc", "f1_max"],
    accelerator="auto",
    devices=1,
)

# Fit the model (build the memory bank from normal images).
print("Building PatchCore memory bank...")
engine.fit(model=model, datamodule=datamodule)

# Test the model on the test set.
print("\nTesting model and evaluating metrics...")
test_results = engine.test(model=model, datamodule=datamodule)

In [ ]:
print("\n--- Performance Evaluation Metrics ---")
for metric_name, value in test_results.items():
    if "auroc" in metric_name or "pro" in metric_name or "f1" in metric_name:
        print(f"{metric_name}: {value:.4f}")

In [ ]:
# Create a test dataloader to get some sample images
test_dataloader = datamodule.test_dataloader()
sample_batch = next(iter(test_dataloader))
sample_batch = {k: v.to(device) for k, v in sample_batch.items()}

# Run inference on the sample batch
model.eval()
with torch.no_grad():
    predictions = model.predict(sample_batch)

# Find an anomalous and a normal image in the batch for visualization
anomalous_indices = (predictions["label"] == 1).nonzero(as_tuple=True)[0]
normal_indices = (predictions["label"] == 0).nonzero(as_tuple=True)[0]

# Helper function to display results
def display_results(predictions, image_index, title):
    item = {key: val[image_index] for key, val in predictions.items()}
    image = item["image"]
    gt_mask = item["gt_mask"]
    anomaly_map = item["anomaly_map"]

    # Get a visualized overlay image from anomalib - Commented out due to ImportError in the previous cell
    # vis_image = get_visualization_image(image.cpu(), anomaly_map.squeeze().cpu(), gt_mask=gt_mask.squeeze().cpu())

    # Plotting
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    # Original Image
    ax[0].imshow(np.transpose(image.cpu().numpy(), (1, 2, 0)))
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    # Ground Truth Mask
    ax[1].imshow(gt_mask.squeeze().cpu().numpy(), cmap='gray')
    ax[1].set_title("Ground Truth Mask")
    ax[1].axis("off")

    # Anomaly Heatmap without overlay as get_visualization_image is not available
    ax[2].imshow(anomaly_map.squeeze().cpu().numpy(), cmap='hot') # Displaying raw anomaly map
    ax[2].set_title("Anomaly Heatmap")
    ax[2].axis("off")

    fig.suptitle(title, fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Display visualizations
if anomalous_indices.numel() > 0:
    display_results(predictions, anomalous_indices[0], "Anomalous Sample Visualization")
else:
    print("No anomalous samples in the first batch.")

if normal_indices.numel() > 0:
    display_results(predictions, normal_indices[0], "Normal Sample Visualization")
else:
    print("No normal samples in the first batch.")